## Assignment 6 - Unit Test Your Functions!

Follow the instructions in the following notebook cell. You will be graded on:

    - Whether your unit tests are passing
    - The completion of this notebook

In [ ]:
# import the stuff you need here: os, glob, natsort, numpy, matplotlib.pyplot, pandas

## Data I/O:

First, in the `utils.py` file from the `functions` folder, recreate the function to get all the `.csv` data files:

```python
def get_files(path_pattern):
    Given a `path_pattern` describing the path to all your data files, sorts the matching data files in alphanumerical order and returns a list with all file paths.

    Input: 
        path_pattern (list): Path to all data files as described by a wildcard (*)
    Returns:
        files (list): Alphanumerical sorted list of all data files matching your path_pattern description.
```

Import it using relative imports and use it in the following cells to load your data:

In [1]:
# first import your utility function: 

# Provide the wildcard (*) string pattern for your data files:
data_pattern = 

# Use your utilities function to read and return a list of sorted data files: 
data_files = get_files()
print('There are {} data csv files in the data folder'.format(len(data_files)))


## Create DataFrame:

Use `pandas` to load your `.csv` file data and turn them into a `dataframe`, then display the first 5 entries of your `dataframe`:

In [ ]:
# read in your csv files, concatenate all the csv's one by one into columns, and replace empty entries with 0's
df = 

# assign column numbers to your data frame
df.columns = 

# Show the first 5 entries of your dataframe:
df.


## Realign our dataframes so we match the peak fluorescent signals:

In `functions.utils.py`, add the following functions:

 - `find_middle()` - find the middle index of input data column and returns it
 - `realign_data()` - Center data around maximum or center of shortest column, padding with 0's, returns 1) a new dataframe with realigned columns and 2) the amount each column was shifted

In `tests` folder, create a file named `test_utils.py` and add the following unit tests functions:

 - `test_find_middle()` - 
 - `test_realign_center()` - 
 - `test_realign_max()` - 

Use your terminal to test these functions with `pytest assignment/tests`. Once all your tests are passing, import the functions and use them in the following cells:

In [ ]:
# realign your data around its maximum or center, whichever one you like better: 
df_aligned, shifts = realign_data()

print('My realigned data frame has shape {}'.format(df_aligned.shape))

## Normalize signal and visualize:

Now we can normalize each column between 0-1 using `scikitlearn`'s `minmaxscale` function and visualize them. <em> In your plots, properly label all axis with appropriate name description and units:

In [ ]:
# Normalize every column entry of your realigned dataframe:

# pre-allocate an empty dataframe of 0's to be filled with normalized values, the index and column of this dataframe will be the same as the df_aligned dataframe:
df_normalized = 

# fill in the `df_normalized` dataframe with normalized columns:
for column, vals in df_aligned.iteritems():
    # find the indices of the column that are not 0, only normalize those:
    ind = df_aligned.index[df_aligned[column]!=0].tolist()
    # Normalize of those:
    df_normalized


In [ ]:
# Use matplotlib to plot all the fluorescent signals:
plt.plot()

In [ ]:
# Now instead of all signals, plot the average signal:
# use numpy to find the mean first:
mean_signal = np.zeros(df_aligned.shape[0])
for index, row in df_aligned.iterrows():
    val = row[row != 0] 
    mean_signal[index] = np.mean(val)

#plot
plt.plot()

In [ ]:
# Finally visualize your aligned, and normalized fluorescent data in a heatmap:
plt.

Lastly, push your submission branch, the `pytests` should automatically be executed upon your push. Make sure these tests are passing!